In [1]:
import numpy as np
from pynq import allocate
from pynq import Overlay
import numpy as np
from numpy import zeros, linspace, fix
from matplotlib.pyplot import figure, imshow, xlabel, ylabel, colorbar, show

ol = Overlay("design_1_wrapper.bit") # hwh is parsed here
ol?

def to_signed(val,b):
    signedVal = val-(2**b)*int(str((val)>>b-1))
    return signedVal

In [2]:
dma = ol.axi_dma

In [3]:
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In [4]:
#https://gist.github.com/parvinderandroid/229438338aea45e7ae8d0929b02a9ff9
def decToBin(number, base, precision):
	number = str(number)
	integerPart = int( number[ : number.index(".") ] )
	fractionalPart = float( number[ number.index(".") : ] )
	
	output = ""
	
	while integerPart != 0:
		output = str( integerPart % base ) + output
		integerPart //= base
	
	if fractionalPart == 0:
		return output
	
	output += "."
	
	while fractionalPart != 0 and precision != 0 :
		fractionalPart *= base
		fractionalPartString = str(fractionalPart)
		output += fractionalPartString[ : fractionalPartString.index(".") ]
		fractionalPart = float( fractionalPartString[ fractionalPartString.index(".") : ] )
		precision -= 1
	
	return output

In [112]:
def binaryString(num):
    whole = fix(num)
    fractional = num - whole
    print("inside binary function!!!")
    
    wholeBin = decToBin(whole, 2, 6)
    wholeBin = wholeBin.zfill(6)
    fractionalBin = decToBin(fractional, 2, 10)
    fractionalBin = fractionalBin.ljust(11, '0')
    print ()
    
    return np.uint64(wholeBin + fractionalBin[1:])

In [141]:
def mandelbrot(CRe, CIm, max_iter, resolution, row_index, result):
    print("Inside second function!!!")
    #Instantiate all variables at start of function
    iterCount = 0
    NMA = 1
    data_size = resolution[1]
    pixels = [(0, 0, 0)]*data_size #Structured to store (ZRe, ZIm, mag) for each pixel
    print("Components instantiated!!!")
    
    #Clear space for input/output buffer
    input_buffer = allocate(shape=(data_size,), dtype=np.uint64)
    output_buffer = allocate(shape=(data_size,), dtype=np.uint64)
    print("Buffers allocated!!!")
    
    #Fill input buffer from real values each iteration, beginning with 0 Z values
    for i in range(data_size):
        print("inside the dodgy loop!!!")
        input_buffer[i] = int(str(binaryString(pixels[i][0]))
                           + str(binaryString(pixels[i][1]))
                           + str(binaryString(CRe[i]))
                           + str(binaryString(CIm[i])), 2)
        
    print("input_buffer all sorted!!!")
    print(input_buffer)
    for i in range(max_iter):
        #Send and receive the row of pixels to and from the hardware
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        
        #Format output_buffer into array of tuples in structure (ZRe, ZIm, mag)
        for i in range(data_size):
            pixelTuple = (output_buffer[i][16:31], output_buffer[i][32:47], output_buffer[i][48:63])
            pixels[i] = pixelTuple

        #Check any pixels escaped this iteration and update result if so
        for j in range(data_size):
            if pixels[j][2] >= 2: #condition to find if pixel magnitude has surpassed 2
                n = j
                
                while result[row_index, n] != 0: #Keep moving through result pixels until one is free
                    n += 1                       #This should work to maintain the bit positions in result
                    
                result[row_index, n] = iterCount
                pixels.pop(j) #remove the pixel from the stream to send into the hardware
                input_buffer.pop(j)
                
        iterCount+=1

    #Sets all of the first row of result that hasn't been changed from 0, to max_iter after the final iteration
    for i in range(data_size):
        if result[row_index, i] == 0:
            result[row_index, i] = max_iter

    return result

In [142]:
def generate_mandelbrot(region, resolution, max_iter):
    print("Inside first function!!!")
    rows, columns = resolution
    CRe = [0]*rows
    CIm = [0]*columns
    result = zeros([rows, columns])
    
    #Splits the specified region into an x by y grid, storing coordinate values in CRe and CIm arrays
    min_Re, max_Re, min_Im, max_Im = region
    for row_index, Re in enumerate(linspace(min_Re, max_Re, num=rows)):
        if Re < 0:
            Re = (Re-(2*Re))
        CRe[row_index] = Re
        for column_index, Im in enumerate(linspace(min_Im, max_Im, num=columns)):
            if Im < 0:
                Im = (Im-(2*Im))
            CIm[column_index] = Im
            
    for row_index in CRe:
        result = mandelbrot(CRe, CIm, max_iter, resolution, row_index, result)
            
    return result

In [143]:
def plot_mandelbrot(result, region):
    figure(dpi=250)
    imshow(result.T, cmap='jet', interpolation='bilinear', extent=region)
    xlabel('Re')
    ylabel('Im')
    colorbar()
    show()

In [144]:
resolution = (10, 10)         #Number of pixels in image (detail of image)

In [145]:
max_iter = 25                   #Number of iterations before pixel is classed 'stable'

In [146]:
region = (-2, 1, -1, 1)           #Entire region of mandelbrot diagram
#region = (-0.77, -0.73, 0.1, 0.14)
#region = (-0.75, -0.745, 0.1, 0.110)
#region = (-0.747, -0.744, 0.107, 0.110)

In [147]:
result = generate_mandelbrot(region, resolution, max_iter)

Inside first function!!!
Inside second function!!!
Components instantiated!!!
Buffers allocated!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside the dodgy loop!!!
inside binary function!!!
inside binary function!!!
inside binary function!!!
inside binary function!!

IndexError: invalid index to scalar variable.

In [ ]:
plot_mandelbrot(result, region)